In [3]:
import sys
sys.path.append('../src/')
sys.dont_write_bytecode = True  # __pycache__ 生成を防ぐ
from vege_train_memo import Model, Experiments


# default
import numpy as np
import pandas as pd

# Manage experiments
import mlflow

# For training
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# import model framework
import lightgbm as lgb


# mlflow setting
DB_PATH = '../server/mlruns.db'
ARTIFACT_LOCATION = '../data/'
EXPERIMENT_NAME = '02_model_train'


In [4]:
# データの読み込み
all_df = pd.read_csv('../data/model_input_data.csv')  # , index_col=0)


grouping_by = 'kind'
target_vege_type = set(all_df[grouping_by])

Y_column_name = 'mode_price'
X_column_name = set(all_df.head().select_dtypes(
    float).columns) - set(Y_column_name)

for vege_type in target_vege_type:
    # 特定の野菜について取り出す
    df = all_df[all_df[grouping_by] == vege_type].select_dtypes(
        float).astype('float32')
    # 'mode_price'に欠損がない部分を訓練データとして利用する
    train = df[~df[Y_column_name].isna()]

    # 目的変数の切り分け
    X = train[X_column_name].to_numpy()
    Y = train[Y_column_name].to_numpy()

    #X_train, X_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2)

## 元コード（メモ）

In [7]:
# pytorch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch


# device setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:


class BiLSTMEncoder(nn.Module):
    def __init__(self, embedding_dim, lstm_dim, vocab_size):
        super(BiLSTMEncoder, self).__init__()
        
        
        self.lstm_dim = lstm_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # bidirectional=Trueでお手軽に双方向のLSTMにできる
        self.bilstm = nn.LSTM(embedding_dim, lstm_dim, batch_first=True, bidirectional=True)

    def forward(self, text):
        embeds = self.word_embeddings(text)

        # 各隠れ層のベクトルがほしいので第１戻り値を受け取る
        out, _ = self.bilstm(embeds)

        # 前方向と後ろ方向の各隠れ層のベクトルを結合したままの状態で返す
        return out


class SelfAttention(nn.Module):
  def __init__(self, lstm_dim, da, r):
    super(SelfAttention, self).__init__()
    self.lstm_dim = lstm_dim
    self.da = da
    self.r = r
    self.main = nn.Sequential(
        # Bidirectionalなので各隠れ層のベクトルの次元は２倍のサイズになってます。
        nn.Linear(lstm_dim * 2, da), 
        nn.Tanh(),
        nn.Linear(da, r)
    )
  def forward(self, out):
    return F.softmax(self.main(out), dim=1)




class SelfAttentionClassifier(nn.Module):
  def __init__(self, lstm_dim, da, r, tagset_size):
    super(SelfAttentionClassifier, self).__init__()
    self.lstm_dim = lstm_dim
    self.r = r
    self.attn = SelfAttention(lstm_dim, da, r)
    self.main = nn.Linear(lstm_dim * 6, tagset_size)

  def forward(self, out):
    attention_weight = self.attn(out)
    m1 = (out * attention_weight[:,:,0].unsqueeze(2)).sum(dim=1)
    m2 = (out * attention_weight[:,:,1].unsqueeze(2)).sum(dim=1)
    m3 = (out * attention_weight[:,:,2].unsqueeze(2)).sum(dim=1)
    feats = torch.cat([m1, m2, m3], dim=1)
    return F.log_softmax(self.main(feats)), attention_weight